In [ ]:
def computeSolidAngle( xp, yp, xh1, yh1, xh2, yh2 ):
    vector1 = np.array( [ xh1-xp, yh1-yp ])
    vector2 = np.array( [ xh2-xp, yh2-yp ])
    vnorm1 = np.linalg.norm( vector1 )
    vnorm2 = np.linalg.norm( vector2 )
    solidAngle = np.arccos( np.dot( vector1, vector2 ) / vnorm1 / vnorm2 )
#     if np.isnan( solidAngle ):
#         solidAngle = pi / 16
    return solidAngle/2/pi

def computeGreenNormal2D( x, y, xj, yj , normalx, normaly ):
    rj2 = (x - xj)**2 + (y - yj)**2
    dGdr = ( 1 / 2 / pi ) / rj2 
    gradrn = normalx * (x - xj) + normaly * (y - yj)
    return -dGdr * gradrn

def trapezoidalQuadRegularized( X, Y, N, U0, AOA ):
    ## check if the geometry has sharp trailing edge
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        tck = createPeriodicSplines( X, Y )
    x, y = computeNodeLocations( N+1, tck )
    theta = np.arctan2( y[:], x[:] )
    ds_dt = computeNormParameterizationDerivative( N+1, tck )
    nx, ny = computeNormal( N+1, tck )
    binormal = computeBinormal( N+1, tck )

    ## correct normal direction
    nx = binormal * nx
    ny = binormal * ny
    
    ## generate system of equations
    A = np.identity( N + 1 )
    b = np.zeros( N + 1 )
    dt = 1/ N

    tol = 1e-12
    for i in range( N ):
        b[i] = U0 * ( x[i] * np.cos( AOA * pi / 180 ) +  y[i] * np.sin( AOA * pi / 180 ) )
        for j in range( N ):
            ## using trapezoidal rule
            
            gij1 = computeGreenNormal2D( x[i], y[i], x[j], y[j], nx[j], ny[j] ) * ds_dt[j] * dt / 2
            gij2 = computeGreenNormal2D( x[i], y[i], x[j+1], y[j+1], nx[j+1], ny[j+1] ) * ds_dt[j+1] * dt / 2
            cond1 = np.abs( gij1 ) > tol and i != j
            cond2 = np.abs( gij2 ) > tol and i != j+1 or ( i != 0 and j == N)
            if cond1 == True or cond2 == True:
                gij1 = computeSolidAngle( x[i], y[i], x[j], y[j], x[j+1], y[j+1] )/2
                gij2 = gij1
                if np.isnan(gij1) == True:
                    gij1 = .25/N
                    gij2 = .25/N

            # first part
            if (i != j):
                A[i, i] = A[i, i] - gij1
                A[i, j] = A[i, j] + gij1
            
            # second part
            if (i != j+1) and not( i ==0 and j+1 == N ): ## if i == j+1, coefficient is not affected 
                A[i, i] = A[i, i] - gij2
                A[i, j+1] = A[i, j+1] + gij2
    
    ## apply periodic boundary conditions for last term
    A[N, 0] = 1
    A[N, -1] = -1
        
    return x, y, theta, A, b, ds_dt